# Control Script

## RUN TRAINING SCRIPT

In [ ]:
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core import Environment
from azureml.core import ScriptRunConfig
from azureml.core import Dataset

In [ ]:
ws = Workspace.from_config()
datastore = ws.get_default_datastore()
dataset = Dataset.File.from_files(path=(datastore, 'datasets/rgbd_dataset'))
# embedding_model = Dataset.File.from_files(path=(datastore, 'embedding_models/bit_m-r50x1_1'))

experiment_name = 'rgbd_model'
experiment = Experiment(workspace=ws, name=experiment_name)

In [ ]:
compute_name = 'gpu-compute-3dcv'

config = ScriptRunConfig(
    source_directory='./src',
    script='train.py',
    compute_target=compute_name,
    arguments=[
        '--data_path', dataset.as_named_input('input').as_mount(),
        ],
)

In [ ]:
# # set up 3dcv environment
# env_path = '.azureml/3dcv-env.yml'
# env = Environment.from_conda_specification(
#     name='3dcv',
#     file_path=env_path
# )
# env.register(workspace=ws);

In [ ]:
env = ws.environments['3dcv']
env.docker.enabled = True
env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04'

# # curated tensorflow environment
# curated_env_name = 'AzureML-TensorFlow-2.2-GPU'
# env = Environment.get(workspace=ws, name=curated_env_name)
# env.docker.enabled = True
# env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04'

In [ ]:
config.run_config.environment = env # or tf_env

In [ ]:
run = experiment.submit(config)
aml_url = run.get_portal_url()
print("Submitted to compute cluster:\n")
print(aml_url)

In [ ]:
run

In [ ]:
# specify show_output to True for a verbose log
run.wait_for_completion (show_output=True) 

In [ ]:
run.get_file_names()

In [ ]:
# register model 
model = run.register_model(model_name='rgbd_model', model_path='outputs/rgbd_model.h5')
print(model.name, model.id, model.version, sep='\t')